In [10]:
%load_ext autoreload
%autoreload 2

from scipy.stats import norm
from scipy.spatial import distance
import numpy as np
from scipy.optimize import linear_sum_assignment
from plotnine import *
import pandas as pd
from tqdm import tqdm
import pyomo.environ as pe
import pyomo.opt as po
from OptimalTransportSolvers import OptimalTransportProblem, HungarianSolver, MinCostFlowSolver, MILPSolver


n_samples=100
kwargs_source={'loc': 0, 'scale': 100}
kwargs_destination={'loc': 0, 'scale': 100, 'size': 2}

source_coordinates = [tuple(norm.rvs(**kwargs_source, size=2)) for i in range(0,n_samples)]
destination_coordinates = [tuple(norm.rvs(**kwargs_destination)) for i in range(0,n_samples)]

cost_matrix = distance.cdist(source_coordinates, destination_coordinates, 'euclidean').astype(int)

x, y = linear_sum_assignment(cost_matrix)

cost_matrix[x, y].sum()


3492

In [11]:
from scipy.stats import norm
from scipy.spatial import distance
import numpy as np
from scipy.optimize import linear_sum_assignment
from plotnine import *
import pandas as pd
from tqdm import tqdm
import pyomo.environ as pe
import pyomo.opt as po
from OptimalTransportSolvers import OptimalTransportProblem, HungarianSolver, MinCostFlowSolver, MILPSolver


n_samples=100
kwargs_source={'loc': 0, 'scale': 100}
kwargs_destination={'loc': 0, 'scale': 100, 'size': 2}

source_coordinates = [tuple(norm.rvs(**kwargs_source, size=2)) for i in range(0,n_samples)]
destination_coordinates = [tuple(norm.rvs(**kwargs_destination)) for i in range(0,n_samples)]

cost_matrix = distance.cdist(source_coordinates, destination_coordinates, 'euclidean').astype(int)

x, y = linear_sum_assignment(cost_matrix)

cost_matrix[x, y].sum()

4501

In [12]:
solver = po.SolverFactory('cbc')

model = pe.ConcreteModel()
source = set(range(0, n_samples))
destination = set(range(0, n_samples))

model.source = pe.Set(initialize=source)
model.destination = pe.Set(initialize=destination)
v=pd.DataFrame(cost_matrix).stack().reset_index().set_index(['level_0','level_1'])[0].to_dict()
model.cost = pe.Param(model.source, model.destination, initialize=v)

model.y = pe.Var(model.source, model.destination, domain=pe.Binary, initialize=0) # variable: 1 if assign parameters set k to city c else 0.

expression = sum(model.cost[c, k] * model.y[c, k] for c in model.source for k in model.destination)
model.obj = pe.Objective(sense=pe.minimize, expr=expression)

def all_served(model, k):
    # assign exactly one parameter set k to every city c.
    constraint = (sum(model.y[c, k] for c in model.source) == 1)
    return constraint

def city_unicity(model, c):
    # assign exactly one parameter set k to every city c.
    constraint = (sum(model.y[c, k] for k in model.destination) == 1)
    return constraint

model.all_served = pe.Constraint(model.destination, rule=all_served)
model.con_unicity = pe.Constraint(model.source, rule=city_unicity)

result = solver.solve(model, timelimit=60, tee=True)
'''solution_df = (pd.Series(model.y.extract_values())
                .reset_index()
                .rename(columns={'level_0': 'city_code', 'level_1': 'candidate', 0: 'y'}))'''

'''solution_df = solution_df.loc[solution_df.y == 1, :]
   solution_df = solution_df.assign(k = n_clusters, objective_function=result['Problem'][0]['Lower bound'], epsilon=epsilon,
                                    epsilon_gap=model.con_epsilon_constraint(), 
                                    termination_condition=result['Solver'][0]['Termination condition'],
                                    termination_message=result['Solver'][0]['Termination message'])'''

print(result['Problem'][0]['Lower bound'])



  0%|          | 0/1000 [01:36<?, ?it/s]


Welcome to the CBC MILP Solver 
Version: 2.7.5 
Build Date: Nov 11 2011 
Revision Number: 1759 

command line - C:\CoinAll-1.6.0-win64-intel11.1\CoinAll-1.6.0-win64-intel11.1\bin\cbc.exe -sec 60 -timeMode elapsed -printingOptions all -import C:\Users\frand\AppData\Local\Temp\tmpv79yajtp.pyomo.lp -stat=1 -solve -solu C:\Users\frand\AppData\Local\Temp\tmpv79yajtp.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 60
Option for timeMode changed from cpu to elapsed
Option for printingOptions changed from normal to all
Presolve 200 (0) rows, 10000 (0) columns and 20000 (0) elements
Statistics for presolved model


Problem has 200 rows, 10000 columns (9998 with objective) and 20000 elements
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 10000 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 200 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of ty

In [13]:
(pd.Series(model.y.extract_values())
                .reset_index()
                .rename(columns={'level_0': 'city_code', 'level_1': 'candidate', 0: 'y'}))

,city_code,candidate,y
0,0,0,0.0
1,0,1,0.0
2,0,2,0.0
3,0,3,0.0
4,0,4,0.0
...,...,...,...
9995,99,95,0.0
9996,99,96,0.0
9997,99,97,0.0
9998,99,98,0.0


In [14]:
n_samples=100
kwargs_source={'loc': 0, 'scale': 1, 'size': 2}
kwargs_destination={'loc': 0, 'scale': 20, 'size': 2}


df = pd.DataFrame([{'wasserstein_metric': MILPSolver(kwargs_source, kwargs_destination, n_samples).solve()} for i in tqdm(range(0, 10**3))])

p = (ggplot(data=df)+
     geom_histogram(aes(x='wasserstein_metric', y=after_stat('density')), bins=100))

print(p)

  0%|          | 0/1000 [00:00<?, ?it/s]

NameError: name 'n_samplestt' is not defined

In [ ]:
'''print('Minimum cost:', model.optimal_cost())
print('')
for i in range(model.num_arcs()):
    if model.flow(i)!=0:
        print(f'{model.tail(i)}->{model.head(i)} \t Flow: {model.flow(i)}')'''

In [ ]:
n_samples=100
kwargs_source={'loc': 0, 'scale': 1, 'size': 2}
kwargs_destination={'loc': 0, 'scale': 20, 'size': 2}


df = pd.DataFrame([{'wasserstein_metric': HungarianSolver(kwargs_source, kwargs_destination, n_samples).solve()} for i in tqdm(range(0, 10**3))])

p = (ggplot(data=df)+
     geom_histogram(aes(x='wasserstein_metric', y=after_stat('density')), bins=100))

print(p)


In [ ]:

n_samples=100
kwargs_source={'loc': 0, 'scale': 1, 'size': 2}
kwargs_destination={'loc': 0, 'scale': 20, 'size': 2}


df = pd.DataFrame([{'wasserstein_metric': MinCostFlowSolver(kwargs_source, kwargs_destination, n_samples).solve()} for i in tqdm(range(0, 10**3))])

p = (ggplot(data=df)+
     geom_histogram(aes(x='wasserstein_metric', y=after_stat('density')), bins=100))

print(p)

In [ ]:
n_samples=100
kwargs_source={'loc': 0, 'scale': 1, 'size': 2}
kwargs_destination={'loc': 0, 'scale': 20, 'size': 2}


df = pd.DataFrame([{'wasserstein_metric': MILPSolver(kwargs_source, kwargs_destination, n_samples).solve(decision_variable_type='binary')} for i in tqdm(range(0, 10**3))])

p = (ggplot(data=df)+
     geom_histogram(aes(x='wasserstein_metric', y=after_stat('density')), bins=100))

print(p)

In [ ]:


class MILPSolver(OptimalTransportProblem):
    
    def __init__(self, 
                 kwargs_source : dict, 
                 kwargs_destination : dict, 
                 n_samples : int) -> None:
        
        super().__init__(kwargs_source, kwargs_destination, n_samples)

    def solve(self, 
              decision_variable_type : str = 'real',
              solver_name : str = 'cbc') -> float:

        if decision_variable_type = 'real':
            decision_domain = pe.NonNegativeReal
        else:
            decision_domain = pe.Binary

        solver = po.SolverFactory(solver_name)
        model = pe.ConcreteModel()

        model.source = pe.Set(initialize=set(range(0, self.n_samples)))
        model.destination = pe.Set(initialize=set(range(0, self.n_samples)))

        cost = pd.DataFrame(self.cost_matrix).stack().reset_index().set_index(['level_0','level_1'])[0].to_dict()
        model.cost = pe.Param(model.source, model.destination, initialize=cost)

        model.y = pe.Var(model.source, model.destination, domain=decision_domain, initialize=0) # variable: 1 if assign parameters set k to city c else 0.

        expression = sum(model.cost[c, k] * model.y[c, k] for c in model.source for k in model.destination)
        model.obj = pe.Objective(sense=pe.minimize, expr=expression)

        def all_served(model, k):
            # assign exactly one parameter set k to every city c.
            constraint = (sum(model.y[c, k] for c in model.source) == 1)
            return constraint

        def city_unicity(model, c):
            # assign exactly one parameter set k to every city c.
            constraint = (sum(model.y[c, k] for k in model.destination) == 1)
            return constraint

        model.all_served = pe.Constraint(model.destination, rule=all_served)
        model.con_unicity = pe.Constraint(model.source, rule=city_unicity)

        result = solver.solve(model, timelimit=60)

        wasserstien_metric = result['Problem'][0]['Lower bound']

        return wasserstein_metric




